# 10.7 Simulations of Galaxy Clusters with the X-IFU

This section contains a tutorial focused on the simulation of observations of Galaxy Clusters with the Athena
X-IFU. Even though we chose a specific type of sources, the simulation approach can be generalized to
any extended source featuring significant spectral variation across the field of view. All files necessary to
run the following simulations are available for download at http://www.sternwarte.uni-erlangen.de/research/sixte/downloads/X-IFU_clusters_tutorial.tgz


In [ ]:
import matplotlib.colors as colors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os
import time
import tempfile

from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import astropy_mpl_style
from datetime import datetime
from funcs import run_comm
from subprocess import check_call, STDOUT
from xspec import Xset, Plot, AllData, ModelManager, Spectrum, Model, AllModels, Fit

plt.style.use(astropy_mpl_style)


In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
xmldir = f"{SIXTE}/share/sixte/instruments/athena-xifu"
xml = f"{xmldir}/xifu_baseline.xml"
AdvXml = f"{xmldir}/xifu_detector_lpa25_tdm_33_275um_20211103.xml"
os.chdir("X-IFU_clusters_tutorial")

## 10.7.1 SIMPUT file for 3D data

The main difficulty of simulating a galaxy cluster observation lies in the building of a suitable SIMPUT file
containing all the information probed by a high-resolution integrated field unit like the X-IFU. To illustrate
this, let us first take a simple point source defined by an XSPEC spectrum with significant line emission
(*xifu_point_source.xcm file*):

In [ ]:
# Simput file for xifu_point_source.xcm (download needed)

xcm = 'xifu_point_source.xcm'
sim_file = 'xifu_point_source.simput'

RA=239.064583333
Dec=66.3470277776
srcFlux=2e-12
Emin = 0.1
Emax = 10.

comm = (f'simputfile Simput={sim_file} RA={RA} Dec={Dec} srcFlux={srcFlux} Emin={Emin} Emax={Emax} ' + 
        f'XSPECFile={xcm} clobber=yes')
#print(comm)
run_comm(comm, 'Creating simput file for point source')

In [ ]:
# Running simulation

evt_file = 'xifu_point_source_evt.fits'
comm = (f'xifupipeline XMLFile={xml} AdvXml={AdvXml} RA={RA} Dec={Dec} ' + 
        f'EvtFile={evt_file} Simput={sim_file} Exposure=100000 clobber=yes')
#print(comm)
run_comm(comm,'Simulating simput file for point source')

In [ ]:
# Create spectrum
spec_file = 'xifu_point_source_spec.pha'
comm = (f"makespec EvtFile={evt_file} Spectrum={spec_file}  EventFilter='GRADING==1' clobber=yes RSPPath={xmldir}") 
#print(comm)
run_comm(comm,'Generating spectrum')

In [ ]:
Plot.device = '/null'
AllData.clear()
AllModels.clear()
s1 = Spectrum(spec_file)
m0 = Model("phabs*apec")
Xset.abund = "wilm"
Xset.cosmo = "70 0. 0.73"
Xset.xsect = "bcmc"
m0.phabs.nH = 0.2
m0.apec.kT = 1.5
m0.apec.Abundanc = 0.75
s1.ignore('**-0.8')
s1.ignore('3.0-**')
Fit.perform()

Plot.device = "/xs"
Plot.xAxis="keV"
Plot("ufs","delchi")
Plot.device = '/null'

### Using existing 2D maps

Use the results obtained by Russell et al. (2012) with Chandra on the galaxy cluster **Abell 2146** for which they could derive temperature and abundance maps (see Figure 21). The brute force approach would be to define a SIMPUT catalog containing one source per input image pixel. If this would be suitable for parameters randomly distributed across the images, astrophysical sources typically feature strong correlations between physical quantities. To take advantage of this property and reduce the amount of sources and spectra in the catalog, the `simputmultispec` tool was developed. It takes as input a series of parameter maps and will create a SIMPUT catalog containing extended sources corresponding to regions with similar parameters according to a given binning.


In [ ]:
# 1st solution: Using existing 2D maps

xcm = 'xifu_point_source.xcm'
sim_file = 'clusterA2146.simput'
img_file = "A2146_SXB_russel_coord_cal.fits"
param_files = "'A2146_kt_russel_coord_cal.fits;A2146_ab_russel_coord_cal.fits'"

RA=239.064583333
Dec=66.3470277776
srcFlux=7.995076796356145e-12
Emin = 0.5
Emax = 10.0
Elow = 0.2
Eup = 12.
comm = (f'simputmultispec Simput={sim_file} XSPECFile={xcm} ImageFile={img_file} ParamFiles={param_files} ' + 
        f'ParamNames="2;3" ParamsLogScale="yes;no" ParamsNumValues="8;8" Emin={Emin} Emax={Emax} RA={RA} Dec={Dec} ' +
        f'srcFlux={srcFlux} Elow={Elow} Eup={Eup} Estep=0.00025 clobber=yes')
#print(comm)
run_comm(comm,'Creating simput file for point source')

#### Running simulation with `xifupipeline`as indicated in **Section 10.7.2**

In [ ]:
# Running simulation with xifupipeline
evt_file = '2d_maps_evt.fits'
comm = (f'xifupipeline XMLFile={xml} AdvXml={AdvXml} RA={RA} Dec={Dec} ' +
        f'EvtFile={evt_file} Simput={sim_file} Exposure=100000 clobber=yes')
#print(comm)
run_comm(comm,'Simulating simput file')

In [ ]:
# modify the EVT file to use later (DS9-type) regions to extract spectra
comm = (f'radec2xy EvtFile={evt_file} refRA={RA} refDec={Dec} Projection=AIT')
#print(comm)
run_comm(comm,f'Adding X,Y coordinates to {evt_file} file')

In [ ]:
#create region files (central and outer)
src_radius = 10. # arcsec radius
core_reg = (239.0655,66.347052,src_radius) # deg
outskirts_reg = (239.03272,66.364708,src_radius) #deg

with open('core.reg', 'w') as f:
    f.write(f'fk5\n')
    f.write(f'circle({core_reg[0]},{core_reg[1]}, {core_reg[2]}") # text={{Core}}\n')

with open('outskirts.reg', 'w') as f:
    f.write(f'fk5\n')
    f.write(f'circle({outskirts_reg[0]},{outskirts_reg[1]},{outskirts_reg[2]}") # text={{outskirts}}\n')
    

In [ ]:
# creating image
img_file = '2d_maps_img.fits'
comm = (f'imgev EvtFile={evt_file} Image={img_file} CoordinateSystem=0 Projection=TAN CUNIT1=deg CUNIT2=deg ' + 
        f'NAXIS1=80 NAXIS2=80 CRVAL1={RA} CRVAL2={Dec} CDELT1=-0.0011888874248538006 CDELT2=0.0011888874248538006 ' +
        f'CRPIX1=40.5 CRPIX2=40.5 clobber=yes')
#print(comm)
run_comm(comm, "Creating image")

In [ ]:
image_data = fits.getdata(img_file, ext=0)
hdu = fits.open(img_file)[0]
wcs = WCS(hdu.header)

fig = plt.figure(figsize=(20,8))

#plot images (left: minimum cut in counts=800; better contrast)
ax = fig.add_subplot(1, 2, 1, projection=wcs)
ax.scatter(core_reg[0], core_reg[1], transform=ax.get_transform('fk5'), s=300,
           edgecolor='C0', facecolor='none', linewidth=2,label='Spec 1')
ax.scatter(outskirts_reg[0], outskirts_reg[1], transform=ax.get_transform('fk5'), s=300,
           edgecolor='white', facecolor='none', linestyle='--',linewidth=2,label='Spec 2')
ax.legend()
cmap = plt.cm.gist_heat
im = ax.imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=800, vmax=5000, clip=True), origin='lower')
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
fig.colorbar(im,ax=ax, pad=0.1)

#plot images (right: minimum cut in counts=1 to see the hexagon)
ax = fig.add_subplot(1, 2, 2, projection=wcs)
ax.scatter(core_reg[0], core_reg[1], transform=ax.get_transform('fk5'), s=300,
           edgecolor='C0', facecolor='none', linewidth=2,label='Spec 1')
ax.scatter(outskirts_reg[0], outskirts_reg[1], transform=ax.get_transform('fk5'), s=300,
           edgecolor='white', facecolor='none', linestyle='--',linewidth=2,label='Spec 2')
ax.legend()
cmap = plt.cm.gist_heat
im = ax.imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=1, vmax=5000, clip=True), origin='lower')
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
fig.colorbar(im,ax=ax, pad=0.1)


In [ ]:
### Create spectrum center
spec_file = 'xifu_center_spec.pha'
comm = (f"makespec EvtFile={evt_file} Spectrum={spec_file} RSPPath={xmldir} EventFilter='regfilter(\"core.reg\")'")
print(comm)
run_comm(comm, 'Generating central spectrum')

In [ ]:
# rebin spectral data
binspec = "xifu_center_spec_rebin.pha"
comm = (f"grppha infile={spec_file} outfile={binspec} clobber=yes comm='group min 20 & exit'")
run_comm(comm, "Rebinning spectrum")

In [ ]:
# Plot central spectrum
Plot.device = '/null'
AllData.clear()
AllModels.clear()

Plot.device = "/xs"
Plot.xAxis="keV"

AllData.clear()
s1 = Spectrum(binspec)
s1.ignore("**-0.5")
s1.ignore("7.0-**")
AllData.ignore("bad")
Plot("ldata")
Plot.device = '/null'

In [ ]:
# Create spectrum outside
spec_file = 'xifu_outside_spec.pha'
comm = (f"makespec EvtFile={evt_file} Spectrum={spec_file} RSPPath={xmldir} EventFilter='regfilter(\"outskirts.reg\")'")
#print(comm)
run_comm(comm,'Generating spectrum')

In [ ]:
# rebin spectral data
binspec = "xifu_outside_spec_rebin.pha"
comm = (f"grppha infile={spec_file} outfile={binspec} clobber=yes comm='group min 20 & exit'")
run_comm(comm, "Rebinning spectrum")

In [ ]:
# Plot outside spectrum
Plot.device = '/null'
AllData.clear()
AllModels.clear()

Plot.device = "/xs"
Plot.xAxis="KeV"

AllData.clear()
s1 = Spectrum(binspec)
s1.ignore("**-0.5")
s1.ignore("8.0-**")
AllData.ignore("bad")
Plot("ldata")
Plot.device = '/null'

### Using galaxy cluster simulations
The second approach will present a SIMPUT tool called `simputmulticell`, specifically designed to provide an interface between SIXTE and 3D data extracted from either toy models or cosmological simulations.

We'll use a 3D dataset saved in a FITS table containing for each point of the grid its position on the sky, temperature, iron abundance and X-ray flux (*xifu_3D_grid.fits*)

We the call `simputmulticell` to construct from this table an optimized SIMPUT catalog that will only contain a limited number of spectra taking advantage of the correlation between the different parameters in a similar way as the `simputmultispec` tool


In [ ]:
# 2nd solution: Using galaxy cluster simulations

xcm = 'wabs_vapec_cosmo.xcm'
sim_file = 'xifu_3D.simput'
param_files = "xifu_3D_grid.fits"

comm = (f'simputmulticell ParamFile={param_files} ParamInputNames="T;FE_ABUND" ParamNames="2;14" InputType=TABLE ' + 
        f'XSPECFile={xcm} ParamsLogScale="no;no" ParamsNumValues="100;100" Estep=0.00025 Emin=0.2 Emax=12.0 ' +
        f'Elow=0.2 Eup=12 clobber=yes Simput={sim_file}')
#print(comm)
run_comm(comm, 'Creating simput file for point source')

#### Running simulation with `xifupipeline`as indicated in **Section 10.7.2**

In [ ]:
# Running simulation
RA = 0
Dec = 0
evt_file = 'gal_clusters_sim_evt.fits'
comm = (f'xifupipeline XMLFile={xml} AdvXml={AdvXml} RA={RA} Dec={Dec} EvtFile={evt_file} ' +
        f'Simput={sim_file} Exposure=1000 clobber=yes')
#print(comm)
run_comm(comm,'Simulating simput file')

In [ ]:
# Creating image
img_file = 'gal_clusters_sim_img.fits'

comm = (f'imgev EvtFile={evt_file} Image={img_file} CoordinateSystem=0 Projection=TAN CUNIT1=deg CUNIT2=deg ' + 
        f'NAXIS1=80 NAXIS2=80 CRVAL1={RA} CRVAL2={Dec} CDELT1=-0.0011888874248538006 CDELT2=0.0011888874248538006 ' + 
        f'CRPIX1=40.5 CRPIX2=40.5 clobber=yes')
#print(comm)
run_comm(comm,'Creating clusters simulation image')

In [ ]:
image_data = fits.getdata(img_file, ext=0)
hdu = fits.open(img_file)[0]
wcs = WCS(hdu.header)

fig = plt.figure(figsize=(12,8))

#plot image
ax = fig.add_subplot(1, 1, 1, projection=wcs)
cmap = plt.cm.gist_heat
im = ax.imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=0.01, vmax=200, clip=True), origin='lower')
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
fig.colorbar(im,ax=ax, pad=0.1)


To reproduce the spectrum of Fig.23 a 1Ms simulation should be performed. After that, command sequence should be as always:

In [ ]:
# Extract spectrum of pixel 1955 using only High Resolution Events
spec_file = 'gal_clusters_sim_spec.pha'
comm = (f"makespec EvtFile={evt_file} Spectrum={spec_file} EventFilter='GRADING==1 && PIXID==1955' " +
        f"clobber=yes RSPPath={xmldir}")
#print(comm)
run_comm(comm,'Generating spectrum')

# rebin spectral data
binspec = "gal_clusters_sim_spec_rebin.pha"
comm = (f"grppha infile={spec_file} outfile={binspec} clobber=yes comm='group min 20 & exit'")
run_comm(comm, "Rebinning spectrum")

#### Visualize the geometry defined in the XML file with `xml2svg` (Fig. 24)

In [ ]:
svg_file = "xifu_baseline.svg"
pdf_file = "xifu_baseline.pdf"
comm = (f"xml2svg XMLFiles={AdvXml} SVGName={svg_file} SVGWidth=3000 Border=10 DrawN=3832 WriteID=yes")
#print(comm)
run_comm(comm, 'Getting chip geometry')

# conver svg file to pdf
run_comm(f"convert {svg_file} {pdf_file}", 'Converting svg to pdf')
run_comm(f"xdg-open {pdf_file}", 'Open the pdf image in the default app')